In [1]:
!pip install -q langchain-google-genai langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [6]:
!pip install -U langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.9 MB/s eta 0:00:00


In [2]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

In [7]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv

In [8]:
class BlogState(TypedDict):

  title:str
  outline:str
  content:str

In [17]:
def create_outline(state:BlogState) -> BlogState:

  title = state['title']
  prompt = f'Generate a detailed outline for a blog on the topic - {title}'

  outline = model.invoke(prompt).content
  state['outline'] = outline

  return state

In [18]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'
    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [19]:
graph = StateGraph(BlogState)

#nodes
graph.add_node('create_outline',create_outline)
graph.add_node("create_blog",create_blog)

#edges
graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog',END)

workflow = graph.compile()

In [20]:
initial_state = {'title':"Rise of AI in India"}
final_state = workflow.invoke(initial_state)
print(final_state)

{'title': 'Rise of AI in India', 'outline': '## Blog Outline: The Rise of AI in India\n\n**I. Introduction (Hook & Context)**\n\n*   **A. Hook:** Start with a compelling statistic, anecdote, or news headline illustrating the growing presence and impact of AI in India.\n    *   Examples:\n        *   "By 2035, AI could add $957 billion to India\'s GDP - McKinsey."\n        *   "A rural Indian farmer is now using AI-powered crop analysis to improve yields."\n        *   "The Indian government is investing heavily in AI research and development."\n*   **B. Briefly Define AI:** Provide a clear and concise definition of Artificial Intelligence for a general audience. Avoid technical jargon.\n*   **C. Contextualize AI in India:** Briefly explain why the rise of AI is particularly relevant and significant for India.\n    *   Mention factors like:\n        *   Large population and demographic dividend\n        *   Growing digital economy\n        *   Need for efficient solutions to address soc

In [24]:
final_state['content']

'## The Rise of AI in India: A Transformative Opportunity\n\n**I. Introduction (Hook & Context)**\n\n*   **A. Hook:** "By 2035, AI could add $957 billion to India\'s GDP," reports McKinsey. This staggering figure underscores the immense potential of Artificial Intelligence to reshape the Indian economy and society. From revolutionizing agriculture to transforming healthcare, AI is rapidly moving from a futuristic concept to a present-day reality.\n\n*   **B. Briefly Define AI:** Artificial Intelligence (AI) refers to the ability of machines to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. It\'s about creating systems that can think and act like humans, but often with greater speed and efficiency.\n\n*   **C. Contextualize AI in India:** The rise of AI is particularly significant for India, a nation grappling with a large population, a burgeoning digital economy, and pressing societal challenges. With its vast pool of da